# ☁️ S3 Integration Tests

Tests de integración específicos para operaciones con Amazon S3.

**Ubicación:** `core/test/integration/03_s3_integration.ipynb`

In [ ]:
# Setup inicial
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta
import json

# Agregar el directorio raíz al path
project_root = Path().absolute().parent.parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Proyecto root: {project_root}")
print(f"☁️ Iniciando tests de S3: {datetime.now()}")

In [ ]:
# Importar servicios necesarios
from core.infraestructure.aws.s3 import S3Client
from core.services.s3_event_service import S3EventService
from core.services.domain_service import DomainService

# Inicializar servicios
s3_client = S3Client()
s3_event_service = S3EventService()
domain_service = DomainService()

print("✅ Servicios S3 inicializados")

# Generar datos únicos para pruebas
test_suffix = datetime.now().strftime("%H%M%S")
print(f"🏷️ Usando sufijo de test: {test_suffix}")

## 1. 🔌 Test de Conexión Básica a S3

In [ ]:
def test_s3_basic_connection():
    """Test de conexión básica a S3"""
    print("🔌 Probando conexión básica a S3...")
    
    try:
        # Intentar conectar
        client = s3_client.conn_s3()
        
        if client is None:
            print("❌ No se pudo establecer conexión a S3")
            print("   Posibles causas:")
            print("   - Credenciales AWS no configuradas")
            print("   - Variables de entorno faltantes")
            print("   - Configuración de región incorrecta")
            return False, "No connection"
        
        print("✅ Conexión a S3 establecida")
        
        # Mostrar configuración
        config = s3_client.config
        print(f"\n📋 CONFIGURACIÓN S3:")
        print(f"   Bucket configurado: {config.get('bucket_name', 'No configurado')}")
        print(f"   Sufijo de archivo: {config.get('file_suffix', 'No configurado')}")
        print(f"   Región: {config.get('region', 'No configurada')}")
        
        return True, "Connected"
        
    except Exception as e:
        print(f"❌ Error de conexión S3: {e}")
        return False, str(e)

s3_connection_status, s3_connection_msg = test_s3_basic_connection()

## 2. 📋 Test de Listado de Buckets

In [ ]:
def test_s3_list_buckets():
    """Test de listado de buckets disponibles"""
    if not s3_connection_status:
        print("⚠️ Saltando test de buckets - No hay conexión S3")
        return False, "No S3 connection"
    
    print("📋 Probando listado de buckets...")
    
    try:
        # Intentar listar buckets
        buckets = s3_client.list_buckets()
        
        if buckets:
            print(f"✅ {len(buckets)} buckets encontrados:")
            
            # Mostrar primeros buckets
            for i, bucket in enumerate(buckets[:5]):
                print(f"   {i+1}. {bucket}")
            
            if len(buckets) > 5:
                print(f"   ... y {len(buckets) - 5} más")
            
            return True, f"{len(buckets)} buckets found"
        else:
            print("⚠️ No se encontraron buckets o no hay permisos para listarlos")
            return False, "No buckets found"
            
    except Exception as e:
        print(f"❌ Error listando buckets: {e}")
        return False, str(e)

buckets_status, buckets_msg = test_s3_list_buckets()

## 3. 📁 Test de Operaciones con Archivos

In [ ]:
def test_s3_file_operations():
    """Test de operaciones básicas con archivos en S3"""
    if not s3_connection_status:
        print("⚠️ Saltando test de archivos - No hay conexión S3")
        return False, "No S3 connection"
    
    print("📁 Probando operaciones con archivos...")
    
    try:
        client = s3_client.conn_s3()
        bucket_name = s3_client.config.get('bucket_name')
        
        if not bucket_name:
            print("❌ No hay bucket configurado")
            return False, "No bucket configured"
        
        print(f"🪣 Usando bucket: {bucket_name}")
        
        # Test 1: Verificar si el bucket existe
        print("\n🔍 1. VERIFICANDO EXISTENCIA DEL BUCKET...")
        try:
            client.head_bucket(Bucket=bucket_name)
            print(f"✅ Bucket '{bucket_name}' existe y es accesible")
        except Exception as e:
            print(f"❌ Bucket '{bucket_name}' no accesible: {e}")
            return False, f"Bucket not accessible: {e}"
        
        # Test 2: Crear un archivo de prueba pequeño
        print("\n📝 2. CREANDO ARCHIVO DE PRUEBA...")
        test_key = f"test/integration_test_{test_suffix}.txt"
        test_content = f"Test file created at {datetime.now().isoformat()}"
        
        try:
            client.put_object(
                Bucket=bucket_name,
                Key=test_key,
                Body=test_content,
                ContentType='text/plain'
            )
            print(f"✅ Archivo de prueba creado: {test_key}")
            
            # Test 3: Leer el archivo creado
            print("\n📖 3. LEYENDO ARCHIVO DE PRUEBA...")
            response = client.get_object(Bucket=bucket_name, Key=test_key)
            content = response['Body'].read().decode('utf-8')
            
            if content == test_content:
                print("✅ Archivo leído correctamente")
            else:
                print("❌ Contenido del archivo no coincide")
            
            # Test 4: Eliminar archivo de prueba
            print("\n🗑️ 4. ELIMINANDO ARCHIVO DE PRUEBA...")
            client.delete_object(Bucket=bucket_name, Key=test_key)
            print(f"✅ Archivo de prueba eliminado: {test_key}")
            
            return True, "File operations successful"
            
        except Exception as e:
            print(f"❌ Error en operaciones de archivo: {e}")
            return False, str(e)
            
    except Exception as e:
        print(f"❌ Error general en test de archivos: {e}")
        return False, str(e)

file_operations_status, file_operations_msg = test_s3_file_operations()

## 4. 📊 Test del Servicio de Eventos S3

In [ ]:
def test_s3_event_service():
    """Test del servicio de eventos S3"""
    if not s3_connection_status:
        print("⚠️ Saltando test de eventos S3 - No hay conexión S3")
        return False, "No S3 connection"
    
    print("📊 Probando servicio de eventos S3...")
    
    try:
        # Crear un dominio de prueba para eventos
        print("\n📝 1. CREANDO DOMINIO DE PRUEBA...")
        test_domain = domain_service.create_domain(
            f"S3_EVENT_TEST_{test_suffix}",
            f"mrn:thing:s3test_{test_suffix}",
            f"mrn:account:s3test_{test_suffix}"
        )
        
        if not test_domain:
            print("❌ No se pudo crear dominio de prueba")
            return False, "Domain creation failed"
        
        domain_id = test_domain['id']
        print(f"✅ Dominio creado: ID {domain_id}")
        
        # Test del servicio de eventos (simulado)
        print("\n📊 2. PROBANDO SERVICIO DE EVENTOS S3...")
        
        # Fechas de prueba
        start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
        end_date = datetime.now().strftime('%Y-%m-%d')
        
        print(f"   Rango de fechas: {start_date} a {end_date}")
        print(f"   Dominio ID: {domain_id}")
        
        # Intentar recuperar eventos (esto puede fallar si no hay archivos)
        result = s3_event_service.retrieve_and_store_events(
            start_date, end_date, domain_id
        )
        
        print(f"\n📋 3. RESULTADO DEL SERVICIO: {result}")
        
        # Interpretar resultados
        if result == "SUCCESS":
            print("✅ Servicio de eventos ejecutado exitosamente")
            service_status = True
        elif result == "ERROR_S3_CONNECTION":
            print("❌ Error de conexión S3 en el servicio")
            service_status = False
        elif result == "INVALID_DOMAIN":
            print("❌ Dominio inválido en el servicio")
            service_status = False
        elif result == "NO_DATA_FOUND":
            print("⚠️ No se encontraron datos (normal en testing)")
            service_status = True  # Esto es normal en testing
        else:
            print(f"⚠️ Resultado desconocido: {result}")
            service_status = False
        
        # Limpiar dominio de prueba
        print("\n🧹 4. LIMPIANDO DOMINIO DE PRUEBA...")
        domain_service.delete_domain(domain_id)
        print(f"✅ Dominio eliminado: {domain_id}")
        
        return service_status, result
        
    except Exception as e:
        print(f"❌ Error en test de eventos S3: {e}")
        return False, str(e)

s3_events_status, s3_events_msg = test_s3_event_service()

## 5. 📊 Resumen Final de Tests S3

In [ ]:
def generate_s3_report():
    """Genera reporte final de tests de S3"""
    print("☁️ REPORTE DE TESTS DE INTEGRACIÓN S3")
    print("=" * 50)
    
    # Resultados de tests
    test_results = {
        "Conexión S3": s3_connection_status,
        "Listado de Buckets": buckets_status,
        "Operaciones de Archivos": file_operations_status,
        "Servicio de Eventos S3": s3_events_status
    }
    
    print("\n🧪 RESULTADOS DE TESTS:")
    successful_tests = 0
    total_tests = len(test_results)
    
    for test_name, success in test_results.items():
        status = "✅ PASS" if success else "❌ FAIL"
        print(f"   {test_name:<25} {status}")
        if success:
            successful_tests += 1
    
    # Calcular tasa de éxito
    success_rate = (successful_tests / total_tests) * 100 if total_tests > 0 else 0
    
    print(f"\n📈 TASA DE ÉXITO: {success_rate:.1f}% ({successful_tests}/{total_tests})")
    
    # Mensajes de estado
    print("\n📋 DETALLES DE ESTADO:")
    print(f"   Conexión: {s3_connection_msg}")
    print(f"   Buckets: {buckets_msg}")
    print(f"   Archivos: {file_operations_msg}")
    print(f"   Eventos: {s3_events_msg}")
    
    # Recomendaciones
    print("\n💡 RECOMENDACIONES:")
    
    if success_rate == 100:
        print("   🎉 ¡Excelente! S3 está completamente operativo.")
        print("   ✅ El sistema puede procesar archivos desde S3.")
    elif success_rate >= 60:
        print("   👍 S3 funciona parcialmente.")
        if not s3_connection_status:
            print("   🔧 Revisar credenciales y configuración de AWS.")
    else:
        print("   🚨 S3 no está funcional.")
        print("   🔧 Revisar configuración completa de AWS S3.")
        print("   📖 Consultar documentación de configuración.")
    
    print(f"\n📋 INFORMACIÓN ADICIONAL:")
    print(f"   Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   Sufijo de test: {test_suffix}")
    
    return success_rate

# Generar reporte final
s3_report = generate_s3_report()

print("\n✅ Tests de integración S3 completados")